# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [20]:
data_1 = data.groupby(['CustomerID','ProductName']).sum()['Quantity'].reset_index()

In [23]:
data_1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [32]:
table = pd.pivot_table(data_1, values='Quantity', index=['ProductName'],columns=['CustomerID'], aggfunc=sum, fill_value=0)

In [203]:
table.describe()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,...,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000
mean,0.139381,0.170354,0.143805,0.148230,0.161504,0.146018,0.170354,0.134956,0.146018,0.148230,...,3.816372,3.595133,3.871681,3.152655,3.982301,3.761062,3.871681,3.871681,4.535398,4.037611
std,0.371427,0.410188,0.369732,0.391338,0.397359,0.377772,0.431269,0.360978,0.377772,0.396963,...,9.001325,9.392041,9.930418,8.794584,10.162958,9.691036,9.502545,10.069003,10.338231,10.072504
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,...,25.000000,50.000000,50.000000,50.000000,75.000000,50.000000,50.000000,50.000000,50.000000,50.000000


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [86]:
m = pdist(table.T, 'euclidean')
squareform(pdist(table.T, 'euclidean'))
similarity = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))),index=table.columns, columns=table.columns)
similarity.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [87]:
similarity_1 = similarity.copy()
similarity_1 = similarity_1.reset_index()

In [88]:
similarity_1.head()

CustomerID,CustomerID,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
1,200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
2,264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
3,356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
4,412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [101]:
lst = similarity_1[['CustomerID',33]]
lst2= lst.sort_values(by = 33, ascending=False)[1:].head()
lst2.set_index('CustomerID', inplace=True)
lst2

CustomerID,33
CustomerID,
3317,0.087047
3535,0.087047
264,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [102]:
data_1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [103]:
lst2= lst2.reset_index()

In [104]:
lst_customers= list(lst2['CustomerID'])
data_filetered=data_1[data_1['CustomerID'].isin(lst_customers)]
data_filetered.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [114]:
data_filetered_2 = data_filetered.groupby(['ProductName']).sum()['Quantity'].reset_index()
data_filetered_2 = data_filetered_2.sort_values(by='Quantity', ascending=False)
data_filetered_2.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [142]:
customer_table = table[33].reset_index()
customer_table_merged = customer_table.merge(data_filetered_2,how = 'outer', on='ProductName').sort_values('Quantity',ascending=False)
customer_table_merged[customer_table_merged[33]==0].head()

,ProductName,33,Quantity
73,Butter - Unsalted,0,3.0
421,Wine - Ej Gallo Sierra Valley,0,3.0
341,Soup - Campbells Bean Medley,0,3.0
415,Wine - Blue Nun Qualitatswein,0,3.0
101,Chicken - Soup Base,0,2.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [143]:
recomendations={}
users = list(set(list(data['CustomerID'])))

In [193]:
def similars(df,user):
    lst = df[['CustomerID',user]]
    lst2= lst.sort_values(by = user, ascending=False)[1:].head()
    return lst2

lst = similars(similarity_1,33)

def filtering(df,lst):
    lst_customers= list(lst['CustomerID'])
    data_filtered=df[df['CustomerID'].isin(lst_customers)]
    return data_filtered

def agreg(df):
    df2 = df.groupby(['ProductName']).sum()['Quantity'].reset_index()
    dfinal = df2.sort_values(by='Quantity', ascending=False)
    return dfinal

def merging(df,df2,i):
    customer_table = df[i].reset_index()
    customer_table_merged = customer_table.merge(df2,how = 'right', on='ProductName').sort_values('Quantity',ascending=False)
    return customer_table_merged[customer_table_merged[i]==0].head()

def creatinglist(df):
    return list(df['ProductName'])

In [194]:
for i in users:
    lst = similars(similarity_1,i)
    df = filtering(data_1,lst)
    dfinal=agreg(df)
    top5 = merging(table,dfinal,i)
    recomendations[i]=creatinglist(top5)

In [195]:
recomendations[200]

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [187]:
df_last=pd.DataFrame.from_dict(recomendations, orient='index').reset_index()
df_last.columns=['UserID','Recomendation 1','Recomendation 2','Recomendation 3','Recomendation 4','Recomendation 5']
df_last.head()

,UserID,Recomendation 1,Recomendation 2,Recomendation 3,Recomendation 4,Recomendation 5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
2,92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
3,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
4,18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [188]:
m = pdist(table.T, 'cosine')
squareform(pdist(table.T, 'cosine'))
similarity_new = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cosine'))),index=table.columns, columns=table.columns)
similarity_new.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [196]:
similarity_new =similarity_new.reset_index()
recomendations2={}

In [197]:
for i in users:
    lst = similars(similarity_new,i)
    df = filtering(data_1,lst)
    dfinal=agreg(df)
    top5 = merging(table,dfinal,i)
    recomendations2[i]=creatinglist(top5)

In [198]:
recomendations2[200]

['Longos - Grilled Salmon With Bbq',
 'Snapple Lemon Tea',
 'General Purpose Trigger',
 'Thyme - Lemon, Fresh',
 'Tomatoes Tear Drop']

In [200]:
recomendations[200]

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']